In [1]:
# %% Load packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import cm
from sklearn.datasets import make_moons, make_circles, make_gaussian_quantiles

from models.load import get_classification_models
from experiments.evaluation import plot_scores, display_stats, plot_ci, plot_ci_all
from datasets.load import load_classification_datasets, print_stats, load_sonar

plt.style.use('ggplot')


In [7]:
# Construct gaussian quantiles dataset
X1, y1 = make_gaussian_quantiles(cov=2.,
                                 n_samples=100, n_features=2,
                                 n_classes=2, random_state=1)
X2, y2 = make_gaussian_quantiles(mean=(3, 3), cov=1.5,
                                 n_samples=100, n_features=2,
                                 n_classes=2, random_state=1)
X_q = np.concatenate((X1, X2))
y_q = np.concatenate((y1, - y2 + 1))
y_q = pd.Series(y_q).map({0:-1,1:1}).to_numpy()

# Moons dataset
X_m, y_m = make_moons(200, random_state=1234,noise=0.1)
y_m = pd.Series(y_m).map({0:-1,1:1}).to_numpy()

# Circles dataset
X_c, y_c = make_circles(200, noise=0.2, factor=0.5, random_state=1)
y_c = pd.Series(y_c).map({0:-1,1:1}).to_numpy()


In [ ]:
models = get_classification_models()[1:]
cmap = cm.get_cmap('Accent', len(models))

In [ ]:
def plot_toy_dataset(X, y, models):
    # and plot
    fig, ax = plt.subplots(figsize=(8,6))

    t_class0 = y == -1
    t_class1 = y == 1
    ax.scatter(X[t_class0, 0], X[t_class0, 1], color='red')
    ax.scatter(X[t_class1, 0], X[t_class1, 1], color='blue')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')

    x1 = np.arange(np.min(X[:,0])*0.9, np.max(X[:,0])*1.1, step=0.05)
    x2 = np.arange(np.min(X[:,1])*0.9, np.max(X[:,1])*1.1, step=0.05)

    xx, yy = np.meshgrid(x1, x2)
    # flatten each grid to a vector
    r1, r2 = xx.flatten(), yy.flatten()
    r1, r2 = r1.reshape((len(r1), 1)), r2.reshape((len(r2), 1))
        
    # horizontal stack vectors to create x1,x2 input for the model
    grid = np.hstack((r1,r2))

    # plot each decision boundary
    i = 0
    model_color = []
    for name, model, _ in models:
        model.fit(X, y) 
        y_hat = model.predict(grid)
        zz = y_hat.reshape(xx.shape)
        color = cmap(i)
        cs = ax.contour(xx, yy, zz, colors=[color])
        
        proxy = plt.Rectangle((0,0),1,1,fc=color)
        model_color.append((name, color, proxy))

        i = i + 1
    
    ax.legend([x[2] for x in model_color],[x[0] for x in model_color])
    fig.show()
    return model_color

In [ ]:
m_color_circles = plot_toy_dataset(X_c,y_c,models)

In [ ]:
m_color_quantile = plot_toy_dataset(X_q,y_q,models)

In [ ]:
m_color_moons = plot_toy_dataset(X_m,y_m,models)

In [2]:
path_class = '/home/victor/git/rna-ppgee/artigo3/report/figures/classification_experiment_chart_data.csv'
save_path = '/home/victor/git/rna-ppgee/artigo3/report'

In [ ]:
plot_ci(path_class, 'Acurácia', 'Modelo')

In [3]:
display_stats(path_class, 'Acurácia', 'std', save_path)

score                                    \
model_name               ELM             GK-H             GK-P   
dataset                                                          
ILPD          70.965 ± 1.889   52.952 ± 6.197   66.289 ± 5.396   
appendicitis  85.818 ± 7.705  76.364 ± 16.318   87.636 ± 8.835   
australian    85.942 ± 5.064   86.087 ± 4.727   87.601 ± 3.491   
banknote      97.959 ± 0.850   91.761 ± 2.124   91.908 ± 2.499   
breastcancer  96.482 ± 1.931   92.093 ± 3.160   94.531 ± 1.292   
bupa          70.454 ± 5.191   50.143 ± 8.406   60.654 ± 3.258   
climate       91.111 ± 1.111   57.593 ± 4.095   91.481 ± 0.907   
fertility     86.000 ± 4.899  52.222 ± 13.966   86.000 ± 4.899   
glass         96.255 ± 3.490  86.845 ± 11.209   97.078 ± 2.265   
haberman      76.237 ± 2.620   53.591 ± 5.800   74.691 ± 2.146   
heart         84.774 ± 2.098   78.189 ± 5.906   82.222 ± 5.926   
ionosphere    88.310 ± 4.149   70.079 ± 6.931   87.002 ± 4.715   
pima          76.422 ± 3.591   67.439 ± 2.860   72.517 ± 4.860   
segmentation  99.654 ± 0.260   57.056 ± 3.457   88.788 ± 1.455   
sonar         65.030 ± 6.330  46.881 ± 15.892  65.926 ± 14.698   

                                                                 \
model_name             GK-RBF           GK-SVM           MLPK-H   
dataset                                                           
ILPD           71.455 ± 0.783   71.431 ± 0.761   60.771 ± 8.558   
appendicitis  84.727 ± 10.122   86.727 ± 9.801  61.545 ± 19.905   
australian     86.087 ± 4.453   85.797 ± 4.798   85.185 ± 3.535   
banknote       99.781 ± 0.334  100.000 ± 0.000  100.000 ± 0.000   
breastcancer   91.208 ± 1.242   98.823 ± 1.184   96.878 ± 0.723   
bupa           61.102 ± 4.445   70.807 ± 8.441   55.958 ± 7.824   
climate        91.111 ± 1.111   91.296 ± 0.849  78.148 ± 13.952   
fertility      87.778 ± 4.157   87.000 ± 4.583  60.000 ± 16.125   
glass          97.684 ± 3.082   96.255 ± 4.089   95.303 ± 4.720   
haberman       72.226 ± 3.560   72.559 ± 2.468  52.570 ± 12.887   
heart          85.185 ± 4.382   79.630 ± 4.757   82.222 ± 5.926   
ionosphere     89.468 ± 6.500   93.730 ± 4.202   92.889 ± 3.634   
pima           75.909 ± 4.156   77.740 ± 2.887   75.518 ± 4.438   
segmentation   98.615 ± 0.794   99.697 ± 0.338   78.163 ± 3.708   
sonar         60.381 ± 17.003   53.899 ± 3.145  69.357 ± 13.203   

                               
model_name             MLPK-P  
dataset                        
ILPD           71.431 ± 0.761  
appendicitis   75.364 ± 7.867  
australian     83.913 ± 3.270  
banknote      100.000 ± 0.000  
breastcancer   96.313 ± 1.990  
bupa           57.096 ± 5.346  
climate        89.815 ± 2.070  
fertility      87.778 ± 4.157  
glass          96.255 ± 3.490  
haberman       72.538 ± 2.255  
heart          87.500 ± 4.512  
ionosphere     93.175 ± 4.046  
pima           76.767 ± 1.870  
segmentation   85.714 ± 0.000  
sonar         66.481 ± 11.236

In [4]:
display_stats(path_class, 'Acurácia', 'ci', save_path)

score                          \
model_name                       ELM                    GK-H   
dataset                                                        
ILPD          70.965 (69.762,72.252)  52.952 (49.158,56.939)   
appendicitis  85.818 (81.091,90.364)  76.364 (66.725,86.002)   
australian    85.942 (82.899,89.420)  86.087 (83.475,88.986)   
banknote      97.959 (97.447,98.542)  91.761 (90.515,93.153)   
breastcancer  96.482 (95.244,97.710)  92.093 (90.000,93.860)   
bupa          70.454 (67.217,73.530)  50.143 (44.772,55.689)   
climate       91.111 (90.556,91.852)  57.593 (54.815,60.000)   
fertility     86.000 (83.000,89.000)  52.222 (43.306,60.000)   
glass         96.255 (94.372,98.550)  86.845 (79.942,94.324)   
haberman      76.237 (74.288,77.970)  53.591 (49.838,57.194)   
heart         84.774 (83.539,86.008)  78.189 (74.486,82.305)   
ionosphere    88.310 (85.976,90.905)  70.079 (66.159,74.730)   
pima          76.422 (74.142,78.572)  67.439 (65.375,69.270)   
segmentation  99.654 (99.481,99.827)  57.056 (54.935,59.048)   
sonar         65.030 (60.833,70.000)  46.881 (37.429,57.143)   

                                                               \
model_name                      GK-P                   GK-RBF   
dataset                                                         
ILPD          66.289 (62.877,69.565)   71.455 (70.986,71.924)   
appendicitis  87.636 (81.907,93.000)   84.727 (78.545,90.916)   
australian    87.601 (85.346,89.698)   86.087 (83.478,88.696)   
banknote      91.908 (90.308,93.378)  99.781 (99.562,100.000)   
breastcancer  94.531 (93.741,95.311)   91.208 (90.312,92.066)   
bupa          60.654 (58.366,62.662)   61.102 (58.207,63.800)   
climate       91.481 (90.926,92.037)   91.111 (90.370,91.852)   
fertility     86.000 (83.000,89.000)   87.778 (85.556,91.111)   
glass         97.078 (95.319,98.295)   97.684 (95.823,99.567)   
haberman      74.691 (73.266,76.116)   72.226 (69.968,74.270)   
heart         82.222 (78.889,85.926)   85.185 (82.593,88.148)   
ionosphere    87.002 (83.827,90.511)   89.468 (85.508,93.429)   
pima          72.517 (69.488,75.418)   75.909 (73.528,78.613)   
segmentation  88.788 (87.834,89.610)   98.615 (98.139,99.091)   
sonar         65.926 (56.717,75.238)   60.381 (50.066,71.168)   

                                                                    \
model_name                       GK-SVM                     MLPK-H   
dataset                                                              
ILPD             71.431 (70.905,71.956)     60.771 (55.492,66.030)   
appendicitis     86.727 (81.000,92.455)     61.545 (49.452,75.275)   
australian       85.797 (83.040,88.986)     85.185 (82.931,87.440)   
banknote      100.000 (100.000,100.000)  100.000 (100.000,100.000)   
breastcancer     98.823 (98.037,99.610)     96.878 (96.484,97.264)   
bupa             70.807 (65.387,75.815)     55.958 (51.007,60.723)   
climate          91.296 (90.741,91.667)     78.148 (70.546,87.407)   
fertility        87.000 (84.000,90.000)     60.000 (49.975,69.025)   
glass            96.255 (93.896,99.048)     95.303 (92.489,98.182)   
haberman         72.559 (71.084,74.086)     52.570 (44.404,60.176)   
heart            79.630 (77.037,82.602)     82.222 (78.519,85.935)   
ionosphere       93.730 (91.175,96.587)     92.889 (90.635,95.175)   
pima             77.740 (75.826,79.617)     75.518 (72.881,78.124)   
segmentation     99.697 (99.481,99.913)     78.163 (75.612,80.423)   
sonar            53.899 (51.845,56.280)     69.357 (61.022,77.054)   

                                         
model_name                       MLPK-P  
dataset                                  
ILPD             71.431 (70.905,71.956)  
appendicitis     75.364 (71.273,80.364)  
australian       83.913 (81.884,85.801)  
banknote      100.000 (100.000,100.000)  
breastcancer     96.313 (95.085,97.538)  
bupa             57.096 (53.182,60.198)  
climate          89.815 (88.519,91.296)  
fertility        87.778 (8